In [9]:
import requests
import json
import pymongo as py
import numpy as np
import pandas as pd
import nltk
from pymongo import MongoClient
from tqdm import tqdm

In [38]:
CA = []
# Extracting the Json files to get Restaurants with no null values but are in California 
with open('yelp_academic_dataset_business.json', 'r') as file:
    for line in tqdm(file, desc='Extracting businesses'):
        x = json.loads(line)
        if x is not None and 'categories' in x and x['categories'] is not None and 'Restaurants' in x['categories'] and 'CA' in x['state']:
            CA.append(x)

Extracting businesses: 150346it [00:01, 84765.40it/s]


In [41]:
CA

[{'business_id': 'IDtLPgUrqorrpqSLdfMhZQ',
  'name': 'Helena Avenue Bakery',
  'address': '131 Anacapa St, Ste C',
  'city': 'Santa Barbara',
  'state': 'CA',
  'postal_code': '93101',
  'latitude': 34.4144445,
  'longitude': -119.6906718,
  'stars': 4.0,
  'review_count': 389,
  'is_open': 1,
  'attributes': {'RestaurantsTakeOut': 'True',
   'NoiseLevel': "u'average'",
   'Caters': 'True',
   'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}",
   'RestaurantsReservations': 'False',
   'BusinessAcceptsCreditCards': 'True',
   'RestaurantsTableService': 'False',
   'GoodForKids': 'True',
   'RestaurantsPriceRange2': '2',
   'WheelchairAccessible': 'True',
   'OutdoorSeating': 'True',
   'RestaurantsDelivery': 'None',
   'HasTV': 'False',
   'RestaurantsAttire': "'casual'",
   'Alcohol': "u'none'",
   'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True

In [43]:
# Holding out the businessid in order to use as a join and to extract reviews with the 
# corresponding to existing CA businesses
businessids = []
for x in CA:
    businessids.append(x['business_id'])

In [44]:
businessids

['IDtLPgUrqorrpqSLdfMhZQ',
 'SZU9c8V2GuREDN5KgyHFJw',
 '4xhGQGdGqU60BIznBjqnuA',
 'ifjluUv4VASwmFqEp8cWlQ',
 'VeFfrEZ4iWaecrQg6Eq4cg',
 'bdfZdB2MTXlT6-RBjSIpQg',
 'xwSWtJcQkzTF6HNm_IMgcg',
 '18eWJFJbXyR9j_5xfcRLYA',
 'vLT1KtrA9bWvjFOg-0xVIg',
 'kHOXo-IZSpixpctmHsGlyA',
 'UFpCraqzFBAhtZqmxmiWsA',
 'Hqz96v1ymucUKNzIWfEKXw',
 'JWFpjvCc_nkNDVtMPx1ZGg',
 '9L-MR0arflwFMF9szEBOOg',
 'uFF40n9pOqHK1ciajdoSEw',
 'aYMpjij5ShtEoZueMrQPRw',
 'GgcvRnt5_z3NEC0D6vNncQ',
 'N-CBZKO37y_WSUN7ZMNGkg',
 'nTZuJSfwCoNOyA1yS3rbOQ',
 'MiMRlsXOpHJMSTDAuw8s1w',
 'V4lqQdll3_TAEtkrHDYbAA',
 'lTVHJAvtFQbQb6cPstEXyA',
 'Ey0t4U6yqtxCeMaGPf7GcA',
 'ju4YP8SLdR_BmWr_-Xh83Q',
 'qdpdUjdkEUpzYDdce2yZng',
 'bjhCtlYHrkgA5Ku8l-rB3g',
 'cbkrO0zra1OiNGvtjw2igQ',
 '_v3DcLatG70adfYzWTd-CQ',
 'SOl2m-ZsAmnCuQGdMZeANQ',
 'SHk_eYKzIOPvgwvTThtXiQ',
 'j-lxie7nEh3UeUTpiqekkQ',
 'sjNVeNqhDjEFHoOybnt2ug',
 'jW4a3W9-UkFxpglHvo_Kqg',
 'bbEXAEFr4RYHLlZ-HFssTA',
 'q6661I3CGW0UB740E5Zjfw',
 '1FURjeGJi_LBXcJQg8eskw',
 'uI15YInpDsGBUZcNfm_TfQ',
 

In [46]:
# Extracting reviews from the yelp review json dataset with a filter to only extract reviews
# that are in the business ids.
filtered_reviews = []

with open('yelp_academic_dataset_review.json', 'r') as file:
    for line in tqdm(file, desc='Extracting reviews'):
        x = json.loads(line)
        if x['business_id'] in businessids:
            filtered_reviews.append(x)

Extracting reviews: 6990280it [03:43, 31342.39it/s]


In [47]:
filtered_reviews

[{'review_id': 'pUycOfUwM8vqX7KjRRhUEA',
  'user_id': '59MxRhNVhU9MYndMkz0wtw',
  'business_id': 'gebiRewfieSdtt17PTW6Zg',
  'stars': 3.0,
  'useful': 0,
  'funny': 0,
  'cool': 0,
  'text': "Had a party of 6 here for hibachi. Our waitress brought our separate sushi orders on one plate so we couldn't really tell who's was who's and forgot several items on an order. I understand making mistakes but the restaraunt was really quiet so we were kind of surprised. Usually hibachi is a fun lively experience and our  cook  said maybe three words, but he cooked very well his name was Francisco. Service was fishy, food was pretty good, and im hoping it was just an off night here. But for the money I wouldn't go back.",
  'date': '2016-07-25 07:31:06'},
 {'review_id': 'eCiWBf1CJ0Zdv1uVarEhhw',
  'user_id': 'OhECKhQEexFypOMY6kypRw',
  'business_id': 'vC2qm1y3Au5czBtbhc-DNw',
  'stars': 4.0,
  'useful': 0,
  'funny': 0,
  'cool': 0,
  'text': "Yes, this is the only sushi place in town. However, it 

In [57]:
# Creating first table/dataframe to extract features for business
owners1 = []
for document in tqdm(CA, desc='Inserting businesses'):
    name = document['name']
    restaurant_rating = document['stars']
    city = document['city']
    business_id = document['business_id']
    longitude = document['longitude']
    latitude = document['latitude']
    categories = document['categories']
    hours = document['hours']
    address = document['address']
    state = document['state']
    owners1.append([name,city,restaurant_rating, business_id, longitude, latitude,categories,hours, address, state])
    
CAbusiness = pd.DataFrame(owners1,columns = ['name','city','restaurant_rating', 'business_id','longitude', 'latitude','categories','hours', 'address', 'state'])

Inserting businesses: 100%|█████████████| 1161/1161 [00:00<00:00, 221133.78it/s]


In [61]:
# Download CAbusiness to a csv file
CAbusiness.to_csv('CAbusiness.csv')

In [59]:
# Creating a CAreviews dataframe that extracts data from 
reviews1 = []

for document in tqdm(filtered_reviews, desc='Inserting businesses'):
    customer_rating = document['stars']
    text = document['text']
    business_id = document['business_id']
    reviews1.append([customer_rating,text,business_id])
    
CAreviews = pd.DataFrame(reviews1, columns = ['customer_rating','text','business_id'])

Inserting businesses: 100%|█████████| 211748/211748 [00:00<00:00, 515036.11it/s]


In [62]:
# Converting the second df/table into a csv file for further exploration
CAreviews.to_csv('CAreviews.csv')